In [3]:
import pandas as pd
import os
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents import AgentType
# -------------------------------------------------------------
# 1. CSV 불러오기
# -------------------------------------------------------------
# traffic.csv에는 날짜, 교통수단, 노선번호, 역/정류장명, 승차/하차 인원 정보 존재
df = pd.read_csv("../data/traffic.csv")


In [4]:
df

,날짜,교통수단,노선번호,역/정류장명,승차인원,하차인원
0,2023-09-01,지하철,1호선,홍대입구,581,453
1,2023-09-02,버스,400,강남역,162,510
2,2023-09-03,버스,200,강남역,220,567
3,2023-09-04,버스,300,강남역,97,525
4,2023-09-05,지하철,4호선,강남역,624,427
...,...,...,...,...,...,...
95,2023-09-06,버스,100,잠실역,649,78
96,2023-09-07,버스,400,강남역,465,385
97,2023-09-08,버스,300,홍대입구,400,577
98,2023-09-09,버스,200,건대입구,477,437


In [ ]:
# -------------------------------------------------------------
# 2. LLM 설정
# -------------------------------------------------------------
# gpt-4o-mini를 사용 (필요 시 다른 모델 가능)
os.environ["OPENAI_API_KEY"] = ""
class PatchedChatOpenAI(ChatOpenAI):
    def invoke(self, input, **kwargs):
        # stop 파라미터가 들어가면 제거
        if "stop" in kwargs:
            kwargs.pop("stop")
        return super().invoke(input, **kwargs)

llm = PatchedChatOpenAI(model="gpt-5-nano")


In [6]:
# -------------------------------------------------------------
# 3. Pandas DataFrame Agent 생성
# -------------------------------------------------------------
# 자연어 질문 → Pandas 코드로 변환 → 실행 → 결과 반환할 수 있는 agent 생성

agent = create_pandas_dataframe_agent(
    llm,
    df,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,  # stop 파라미터 최소화
)


In [7]:
# -------------------------------------------------------------
# 4. 질문 실행 (자연어 → Pandas 분석)
# -------------------------------------------------------------
# 예시 질의: "홍대입구에서 버스에 승차한 인원은 총 몇 명인가요?"와 같은 데이터에서 할 수 있는 질문을 작성합니다.
query = "홍대입구에서 버스에 승차한 인원은 총 몇 명인가요?"
answer = agent.invoke(query)


In [8]:
# -------------------------------------------------------------
# 5. 결과 출력
# -------------------------------------------------------------
print("질문:", query)
print("답변:", answer["output"])

질문: 홍대입구에서 버스에 승차한 인원은 총 몇 명인가요?
답변: 0명입니다.

설명: 주어진 데이터에서 홍대입구에서 버스의 승차 인원은 한 행도 없고, 홍대입구에 해당하는 행은 모두 지하철이므로 버스로 승차한 인원 합계는 0입니다. 필요하시면 아래처럼 코드로도 확인할 수 있습니다.

예) pandas 코드
total_bus_boarding_at_hongdae = df[(df['역/정류장명'] == '홍대입구') & (df['교통수단'] == '버스')]['승차인원'].sum()
print(total_bus_boarding_at_hongdae)
